In [1]:
import pandas as pd
from datetime import datetime

In [2]:
NBA = pd.read_csv('data/coaching_changes_basketball.psv', sep='|')

In [3]:
NBA.head()

,date,team,acquired,relinquished,notes
0,1946-06-01,Nationals (NBL),Benny Borgmann,NaN,hired as head coach (date approximate)
1,1946-07-01,Bombers (BAA),Ken Loeffler,NaN,hired as head coach (date approximate)
2,1946-07-01,Capitols (BAA),Arnold Auerbach / Red Auerbach,NaN,"hired as head coach, signed to a 1-year, $5K c..."
3,1946-07-01,Celtics (BAA),Honey Russell,NaN,hired as head coach (date approximate)
4,1946-07-01,Falcons (BAA),Glenn Curtis,NaN,hired as head coach (date approximate)


In [4]:
NBA[NBA.acquired == "Sidney Lowe"]

,date,team,acquired,relinquished,notes
619,1991-07-18,Timberwolves,Sidney Lowe,NaN,hired as assistant coach
682,1993-01-11,Timberwolves,Sidney Lowe,NaN,promoted to interim head coach
687,1993-03-24,Timberwolves,Sidney Lowe,NaN,"pomoted to head coach, signed to a 3-year cont..."
762,1994-10-03,Cavaliers,Sidney Lowe,NaN,hired as assistant coach
1137,1999-08-05,Timberwolves,Sidney Lowe,NaN,hired as assistant coach
1195,2000-06-01,Grizzlies,Sidney Lowe,NaN,"hired as head coach, signed to a reported 4-ye..."
1475,2003-08-13,Timberwolves,Sidney Lowe,NaN,hired as assistant coach
1722,2005-08-12,Pistons,Sidney Lowe,NaN,hired as assistant coach
2233,2011-10-06,Jazz,Sidney Lowe,NaN,hired as assistant coach
2478,2014-06-13,Timberwolves,Sidney Lowe,NaN,hired as assistant coach


In [5]:
NBA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2629 entries, 0 to 2628
Data columns (total 5 columns):
date            2629 non-null object
team            2629 non-null object
acquired        2000 non-null object
relinquished    626 non-null object
notes           2629 non-null object
dtypes: object(5)
memory usage: 123.2+ KB


keep only head coaches

In [6]:
nba_hc = NBA[
    NBA.notes.str.contains('head coach') &
    ~NBA.notes.str.contains('associate')
]
print "{} head coaching transactions in data".format(len(nba_hc))

1129 head coaching transactions in data


In [7]:
nba_hc.tail()

,date,team,acquired,relinquished,notes
2621,2016-01-22,Cavaliers,Tyronn Lue,NaN,promoted to head coach
2624,2016-02-01,Suns,NaN,Jeff Hornacek,fired as head coach
2625,2016-02-01,Suns,Earl Watson,NaN,promoted to interim head coach
2626,2016-02-08,Knicks,NaN,Derek Fisher,fired as head coach
2627,2016-02-08,Knicks,Kurt Rambis,NaN,promoted to interim head coach


keep only head coaches from 1988-89 season on

In [8]:
nba_hc['date'] = pd.to_datetime(nba_hc['date'], format='%Y-%m-%d')

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [9]:
nba_hc = nba_hc[nba_hc.date > datetime(1988,7,1)]

keep only lines that involve a certain transaction

* 'moved': Tim Floyd 99 (was technically coach before season started)
* 'pomoted': Sidney Lowe
* Fratello marked as 'fired / resigned' from Knicks
* In 07 Adelman marked as 'signed' to extension

In [10]:
nba_hc[nba_hc.notes.str.contains('released')]

,date,team,acquired,relinquished,notes
2369,2013-06-25,Celtics,NaN,Glenn Rivers / Doc Rivers,team released head coach from contract


In [11]:
nba_hc[
    # start words
    ~nba_hc.notes.str.contains('hired') &
    ~nba_hc.notes.str.contains('promoted') & 
    ~nba_hc.notes.str.contains('pomoted') &
    ~nba_hc.notes.str.contains('moved') &
    ~nba_hc.notes.str.contains('named') &
    ~nba_hc.notes.str.contains('signed') &
    
    # unwillfully let go
    ~nba_hc.notes.str.contains('fired') &
    ~nba_hc.notes.str.contains('expired') &
    
    # willfully let go
    ~nba_hc.notes.str.contains('retired') &
    ~nba_hc.notes.str.contains('resigned') &
    ~nba_hc.notes.str.contains('released') &
    
    # reassignment
    ~nba_hc.notes.str.contains('replaced') &
    ~nba_hc.notes.str.contains('relinquished') &
    
    # extensions
    ~nba_hc.notes.str.contains('re-signed') & 
    ~nba_hc.notes.str.contains('re-sign') & 
    ~nba_hc.notes.str.contains('extended') &
    ~nba_hc.notes.str.contains('re-signed') & 
    ~nba_hc.notes.str.contains('exercised') &
    
    True
]

,date,team,acquired,relinquished,notes
779,1995-02-20,Nuggets,Gene Littles,NaN,demoted from interim head coach to assistant c...
951,1997-05-08,Celtics,Rick Pitino,NaN,assumed title of president in addition to head...
973,1997-06-17,Pistons,Doug Collins,NaN,restructured contract as head coach
1298,2001-07-25,76ers,Larry Brown,NaN,announced he will return as head coach
1418,2003-06-11,Knicks,second round pick (2005 #54-Dijon Thompson),NaN,received from Rockets as compensation for Knic...
1419,2003-06-11,Rockets,NaN,second round pick (2005 #54-Dijon Thompson),sent to Knicks as compensation for Knicks agre...
1851,2007-06-07,Magic,NaN,NaN,sent to Heat as compensation for head coach St...
1852,2007-06-07,Heat,NaN,NaN,received from Magic as compensation for head c...
2198,2011-06-01,Raptors,NaN,Jay Triano,team declined to exercise contract option on h...
2371,2013-06-25,Clippers,NaN,2015 first round pick (#28-R.J. Hunter),sent to Celtics as compensation for head coach...
